# Dataset CoLA

Primeiramente vamos baixar o pacopte wget para baixar o dataset CoLA:

In [ ]:
!pip install wget

## Carregando o conjunto de dados (CoLA)

Usaremos o conjunto de dados do *Corpus of Linguistic Acceptability (CoLA)* para a classificação de uma única frase. É um conjunto de frases rotuladas como gramaticalmente corretas ou incorretas. Foi publicado pela primeira vez em maio de 2018, e é um dos testes incluídos no “Benchmark GLUE” em que competem modelos como o BERT.

Faça *download* do dataset neste link: https://nyu-mll.github.io/CoLA/ e descompacte o conjunto de dados na mesma pasta em que está trabalhando.

Ou, se estiver no Colab, use o comando abaixo:


In [ ]:
import wget
import zipfile
import os

print('Baixando dataset...')

# URL para baixar o dataset em formato zip.
url = 'https://nyu-mll.github.io/CoLA/cola_public_1.1.zip'

# Caminhos
zip_path = '../cola_public_1.1.zip'
extract_path = '../dataset'

# Baixar o arquivo, se necessário
if not os.path.exists(zip_path):
    wget.download(url, zip_path)

# Extrair, se ainda não foi extraído
if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print('Dataset extraído com sucesso.')
else:
    print('Dataset já extraído.')